# ESIDA DB Data Access Demo for Data Quality

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from urllib.parse import urlencode
import datetime as dt

import pandas as pd
import geopandas

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np

In [ ]:
# Run for local database
HOST = 'http://localhost'
auth = None

In [ ]:
# Run for hosted database
HOST = 'https://esida.informatik.haw-hamburg.de'
auth = HTTPBasicAuth('esida', 'opendata22')

# Get region shapes from database

In [ ]:
# Parameters:
# - type=region|district (optional)
# - wkt (optional, returns Geometry)
q = {
    'type': 'region',
    'wkt': True
}
d = requests.get(f"{HOST}/api/v1/shapes?{urlencode(q)}", auth=auth).json()

# DataFrame
regions_gdf = geopandas.GeoDataFrame(d['data'])
if q['wkt']:
    regions_gdf['geometry'] = geopandas.GeoSeries.from_wkt(regions_gdf['wkt'])
    regions_gdf = regions_gdf.drop(columns=['wkt'])

In [ ]:
regions = dict(zip(regions_gdf.id, regions_gdf.name))
regions_Name2Id = dict(zip(regions_gdf.name, regions_gdf.id))

In [ ]:
regions_gdf.plot()
plt.title('Regions')
plt.axis('off')
plt.show()

In [ ]:
len(regions_gdf)

# Get district shapes from database

In [ ]:
# Parameters:
# - type=region|district (optional)
# - wkt (optional, returns Geometry)
q = {
    'type': 'district',
    'wkt': True
}
d = requests.get(f"{HOST}/api/v1/shapes?{urlencode(q)}", auth=auth).json()

# DataFrame
districts_gdf = geopandas.GeoDataFrame(d['data'])
if q['wkt']:
    districts_gdf['geometry'] = geopandas.GeoSeries.from_wkt(districts_gdf['wkt'])
    districts_gdf = districts_gdf.drop(columns=['wkt'])

In [ ]:
districts = dict(zip(districts_gdf.id, districts_gdf.name))

In [ ]:
districts_gdf.plot()
plt.title('Districts')
plt.axis('off')
plt.show()

In [ ]:
len(districts_gdf)

# Get data layer information

In [ ]:
q = {}
d = requests.get(f"{HOST}/api/v1/parameters?{urlencode(q)}", auth=auth).json()

# DataFrame
datalayers_df = pd.DataFrame(d['data'])

In [ ]:
datalayers_df

In [ ]:
# list all available loaded parameters
loaded_parameters = list(datalayers_df['parameter_id'].values)

# Get map of temporal coverage

In [ ]:
# for which parameters do want to show temporal coverage?
parameters = ['chirts_maxt', 'meteo_maxt']

In [ ]:
cache = {}
for parameter_id in parameters:
    rows = []

    for name, shape_id in regions_Name2Id.items():
        if shape_id not in cache:
            q = {
                'shape_id': shape_id,
            }
            d = requests.get(f"{HOST}/api/v1/parameters?{urlencode(q)}", auth=auth).json()

            cache[shape_id] = pd.DataFrame(d['data'])
        dfx = cache[shape_id]
        row = {
            'id': shape_id,
            'value': dfx[dfx['parameter_id'] == parameter_id].reset_index(drop=True).at[0, 'temporal_coverage']
        }
        rows.append(row)

    dfxv = pd.DataFrame(rows)
    gdfxv = regions_gdf.merge(dfxv, on='id')

    gdfxv.plot(column='value',  vmin=0, vmax=1, legend=True)
    plt.title(f"{parameter_id} between 2010 and 2020")
    plt.show()

# Spatial coverage

**ATTENTION**: the queries regarding the spatial coverage are quite cpu time heavy (minutes up to houres for the large spatial data sets). This can lead to timeout errors due to the Kubernetes Ingress Proxy for the provided hosting of the database on the HAW.

This queries will probably fail in combination with the HAW hosted database. You need to run the ESIDA database locally and import the data of interest and then use the cells below.

In [ ]:
cache = {}
for parameter_id in loaded_parameters[:2]:
    rows = []

    for name, shape_id in regions_Name2Id.items():
        if shape_id not in cache:
            q = {
                'shape_id': shape_id,
                'da_spatial': True
            }
            d = requests.get(f"{HOST}/api/v1/parameters?{urlencode(q)}", auth=auth).json()

            cache[shape_id] = pd.DataFrame(d['data'])
        dfx = cache[shape_id]
        row = {
            'id': shape_id,
            'value': dfx[dfx['parameter_id'] == parameter_id].reset_index(drop=True).at[0, 'spatial_coverage']
        }
        rows.append(row)

    dfxv = pd.DataFrame(rows)
    gdfxv = regions_gdf.merge(dfxv, on='id')

    gdfxv.plot(column='value',  vmin=0, vmax=1, legend=True)
    plt.title(f"{parameter_id} between 2010 and 2020")
    plt.show()

# Spatial coverage for a single parameter

In [ ]:
p = 'worldpop_popd'
q = {}
d = requests.get(f"{HOST}/api/v1/da_spatial/{p}?{urlencode(q)}", auth=auth).json()

df = pd.DataFrame(d['data'])

In [ ]:
# Total spatial completness over all files (mean)
d['data_quality']

In [ ]:
# per file / per shape coverage
df.head(5)

In [ ]:
df.to_csv('da_spatial_worldpop_popd.csv', index=False)

In [ ]:
spatial_coverage = []
for shape_id, name in regions.items():
    dfx = df[df['shape_id'] == shape_id]
    spatial_coverage.append({
        'id': shape_id,
        'coverage': dfx['coverage'].mean()
    })

In [ ]:
spatial_coverage_df = pd.DataFrame(spatial_coverage)

In [ ]:

#fig, ax = plt.subplots(1, 1, figsize=(10,10)) # rows, cols

ax = regions_gdf.merge(spatial_coverage_df, on='id').plot(column='coverage', legend=True, legend_kwds={"fmt": "{:.0f}aa"})
#ax.yaxis.set_major_formatter(mtick.PercentFormatter())



In [ ]:
spatial_coverage = []
for shape_id, name in districts.items():
    dfx = df[df['shape_id'] == shape_id]
    spatial_coverage.append({
        'id': shape_id,
        'coverage': dfx['coverage'].mean()
    })
spatial_coverage_df = pd.DataFrame(spatial_coverage)

In [ ]:
ax = districts_gdf.merge(spatial_coverage_df, on='id').plot(column='coverage', legend=True)
#districts_gdf.plot(ax=ax, facecolor='none', ec='black')